<a href="https://colab.research.google.com/github/martincastanonicolas-source/IA/blob/master/01-numpy-analisis/analisis_meteorologico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Análisis de Datos Meteorológicos con NumPy**


In [64]:
# DATASET
import numpy as np
np.random.seed(42)
n_estaciones, n_dias, n_horas, n_vars = 5, 30, 24, 6
datos = np.zeros((n_estaciones, n_dias, n_horas, n_vars))

# Temperatura base por estación + variación diaria + ruido
temp_base = np.array([8, 12, 14, 10, 6]).reshape(5,1,1)
hora = np.arange(24).reshape(1,1,24)
variacion_diaria = 5 * np.sin((hora - 6) * np.pi / 12)
datos[:,:,:,0] = temp_base + variacion_diaria + np.random.normal(0, 2, (5,30,24))

# Resto de variables (humedad, presión, viento, precipitación, radiación)
datos[:,:,:,1] = np.clip(70 - datos[:,:,:,0] + np.random.normal(0, 10, (5,30,24)), 30, 100)
datos[:,:,:,2] = 1013 + np.random.normal(0, 5, (5,30,24))
datos[:,:,:,3] = np.abs(np.random.normal(15, 10, (5,30,24)))
datos[:,:,:,4] = np.maximum(0, np.random.exponential(2, (5,30,24)))
radiacion_base = np.maximum(0, 400 * np.sin((hora - 6) * np.pi / 12))
datos[:,:,:,5] = np.clip(radiacion_base + np.random.normal(0, 50, (5,30,24)), 0, 800)

##Bloque 1: Exploración y acceso a datos

In [65]:
stations = ["Madrid-Retiro", "Barcelona-El Prat", "Sevilla-Aeropuerto", "Bilbao-Sondica", "Granada-Base Aérea"]

#Ejercicio 1. Verificación de estructura
#Muestra la forma (shape) del array, el número total de mediciones y el tipo de datos (dtype). ¿Cuántas mediciones individuales contiene el dataset?
print(f"Ejercicio 1:")
print(f"Forma de los datos: {datos.shape}")
print(f"Tipo de la colección: {datos.dtype}")
print(f"Número de registros: {datos.size}")


#Ejercicio 2. Extracción de series temporales
#Extrae la serie de temperaturas de Madrid-Retiro durante el día 15. El resultado debe ser un array de 24 elementos.
print(f"\n\nEjercicio 2:")
temps_madrid_15_days = np.round(datos[0, 14, :, 0], 2)
print(f"Temperatura en Madrid-Retiro el día 15 - {temps_madrid_15_days}")


#Ejercicio 3. Comparativa entre estaciones
#Extrae las temperaturas de todas las estaciones a las 12:00 del día 20. El resultado debe ser un array de 5 elementos.
print(f"\n\nEjercicio 3:")
print(f"Temperaturas por estación día 20 a las 12:00")
stations_temps = np.round(datos[:, 19, 11, 0], 2)
for i, station in enumerate(stations):
    print(f"{station}: {stations_temps[i]}°C")


#Ejercicio 4. Bloque de datos
#Extrae todos los datos de Barcelona durante la primera semana (días 0-6). ¿Cuál es la forma del array resultante?
print(f"\n\nEjercicio 4:")
barcelona_first_week = datos[1, 0:7, :, :]
print(f"Forma de los datos de Barcelona en la primera semana: {barcelona_first_week.shape}")

Ejercicio 1:
Forma de los datos: (5, 30, 24, 6)
Tipo de la colección: float64
Número de registros: 21600


Ejercicio 2:
Temperatura en Madrid-Retiro el día 15 - [ 1.35  2.53  4.5   3.34  3.86  7.19  8.49  8.28  9.56 12.    9.43 10.01
 11.56 12.4  12.95 14.49 12.22  8.97  7.96  4.7   5.46  3.89  4.32  1.52]


Ejercicio 3:
Temperaturas por estación día 20 a las 12:00
Madrid-Retiro: 14.34°C
Barcelona-El Prat: 16.38°C
Sevilla-Aeropuerto: 16.58°C
Bilbao-Sondica: 16.23°C
Granada-Base Aérea: 14.01°C


Ejercicio 4:
Forma de los datos de Barcelona en la primera semana: (7, 24, 6)


##Bloque 2: Estadística descriptiva


In [66]:
hours = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00",
         "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00",
         "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"]

#Ejercicio 5. Temperatura media por estación
#Calcula la temperatura media de todo el mes para cada estación. El resultado debe ser un array de 5 elementos. ¿Qué estación es más cálida? ¿Cuál más fría?

print("Ejercicio 5:")
mean_temps_per_station = np.mean(datos[:, :, :, 0], axis=(1, 2))
mean_temps_per_station = np.round(mean_temps_per_station, 2)
mean_temps_per_station = np.sort(mean_temps_per_station)[::-1]

for i, station in enumerate(reversed(stations)):
    print(f"{station}: {mean_temps_per_station[i]}")


#Ejercicio 6. Perfil horario medio
#Calcula la temperatura media de todas las estaciones para cada hora del día (promediando todos los días). El resultado debe ser un array de
#24 elementos que muestre el perfil térmico diario típico.

#Versión sin agrupar horas
#mean_temps_stations_per_hour = np.round(np.mean(datos[:, :, :, 0], axis=(0, 1)), 2)
#for i, hour in enumerate(hours):
#    print(f"{hour}: {mean_temps_stations_per_hour[i]}")

print(f"\nEjercicio 6:")
mean_temps_stations_per_hour = np.mean(datos[:, :, :, 0], axis=(0, 1))
mean_temps_stations_per_hour = np.round(mean_temps_stations_per_hour, 2)

for i in range(0, len(hours), 4):
    linea = "\t".join([f"{hours[j]}: {mean_temps_stations_per_hour[j]}" for j in range(i, min(i + 4, len(hours)))])
    print(linea)


#Ejercicio 7. Variabilidad climática
#Calcula la desviación estándar de la temperatura para cada estación. ¿Qué estación presenta mayor variabilidad térmica?
print(f"\nEjercicio 7:")
std_temps_per_station = np.std(datos[:, :, :, 0], axis=(1, 2))
std_temps_per_station = np.round(std_temps_per_station, 2)
std_temps_per_station = np.sort(std_temps_per_station)[::-1]

for i, station in enumerate(reversed(stations)):
    print(f"{station}: {std_temps_per_station[i]}")


#Ejercicio 8. Extremos meteorológicos
#Encuentra la temperatura máxima y mínima registradas en todo el dataset. Indica en qué estación, día y hora se produjeron
#(pista: usa np.argmax/np.argmin y np.unravel_index).
print(f"\nEjercicio 8:")
max_temp = np.round(np.max(datos[:, :, :, 0]), 2)
idx_max_temp = np.argmax(datos[:, :, :, 0])

min_temp = np.round(np.min(datos[:, :, :, 0]), 2)
idx_min_temp = np.argmin(datos[:, :, :, 0])

max_array = np.unravel_index(idx_max_temp, datos[:, :, :, 0].shape)
min_array = np.unravel_index(idx_min_temp, datos[:, :, :, 0].shape)

print(f"La temperatura máxima fue de {max_temp}°C en la estación {stations[max_array[0]]} el dia {max_array[1]} a las {hours[max_array[2]]}")
print(f"La temperatura mínima fue de {min_temp}°C en la estación {stations[min_array[0]]} el dia {min_array[1]} a las {hours[min_array[2]]}")

Ejercicio 5:
Granada-Base Aérea: 14.05
Bilbao-Sondica: 12.18
Sevilla-Aeropuerto: 10.05
Barcelona-El Prat: 7.98
Madrid-Retiro: 6.0

Ejercicio 6:
00:00: 5.03	01:00: 5.16	02:00: 5.48	03:00: 6.51
04:00: 7.64	05:00: 9.0	06:00: 10.23	07:00: 11.22
08:00: 12.55	09:00: 13.55	10:00: 14.12	11:00: 14.96
12:00: 15.19	13:00: 14.87	14:00: 14.37	15:00: 13.68
16:00: 12.35	17:00: 11.69	18:00: 10.17	19:00: 8.84
20:00: 7.55	21:00: 6.14	22:00: 5.64	23:00: 5.32

Ejercicio 7:
Granada-Base Aérea: 4.11
Bilbao-Sondica: 4.11
Sevilla-Aeropuerto: 4.09
Barcelona-El Prat: 4.06
Madrid-Retiro: 4.02

Ejercicio 8:
La temperatura máxima fue de 25.11°C en la estación Sevilla-Aeropuerto el dia 21 a las 13:00
La temperatura mínima fue de -4.19°C en la estación Granada-Base Aérea el dia 23 a las 22:00


##Bloque 3: Filtrado y selección condicional

In [93]:
#Ejercicio 9. Detección de heladas
#Crea una máscara booleana que identifique todas las mediciones con temperatura
#inferior a 0ºC. ¿Cuántas heladas se registraron en total?
#¿Qué porcentaje del total de mediciones representa?
mascara_heladas = datos[:,:,:,0] < 0
n_heladas = np.sum(mascara_heladas)
total_mediciones = n_estaciones * n_dias * n_horas
porcentaje_heladas = (n_heladas / total_mediciones) * 100
print(f"\nEjercicio 9:")
print(f"Total de heladas: {n_heladas}")
print(f"Porcentaje de heladas: {porcentaje_heladas:.2f}%")

#Ejercicio 10. Días de lluvia significativa
#Identifica los días en los que la precipitación acumulada diaria
#(suma de las 24 horas) superó los 10mm en alguna estación.
#Muestra qué estaciones y qué días.
precip_diaria = np.sum(datos[:,:,:,4], axis=2)
lluvia_significativa = precip_diaria > 10
total_dias_lluvia = np.sum(lluvia_significativa)
estaciones, dias = np.where(lluvia_significativa)
print(f"\nEjercicio 10:")
print(f"Cantidad de días que llovió mas de 10 mm en alguna estación: {total_dias_lluvia}")
for i in range(n_estaciones):
    dias_lluvia = np.where(lluvia_significativa[i])[0] + 1
    if len(dias_lluvia) > 0:
        print(f"{stations[i]}: {', '.join(dias_lluvia.astype(str))}")


#Ejercicio 11. Condiciones de comfort
#Define condiciones de comfot como: temperatura entre 18ºC y 24ºC,
#humedad entre 40% y 60%, y viento inferior a 20 km/h.
#¿Qué porcentaje de mediciones cumplen las tres condiciones simultaneamente?
print("\nEjercicio 11:")
temperatura_comfort = (datos[:,:,:,0] >= 18) & (datos[:,:,:,0] <= 24)
humedad_comfort = (datos[:,:,:,1] >= 40) & (datos[:,:,:,1] <= 60)
viento_comfort = (datos[:,:,:,3] < 20)

comfort_simultaneo = temperatura_comfort & humedad_comfort & viento_comfort
total_mediciones = n_estaciones * n_dias * n_horas
mediciones_comfort = np.sum(comfort_simultaneo)
porcentaje = (mediciones_comfort / total_mediciones) * 100

print(f"Porcentaje: {porcentaje:.2f}%")


Ejercicio 9:
Total de heladas: 62
Porcentaje de heladas: 1.72%

Ejercicio 10:
Cantidad de días que llovió mas de 10 mm en alguna estación: 150
Madrid-Retiro: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30
Barcelona-El Prat: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30
Sevilla-Aeropuerto: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30
Bilbao-Sondica: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30
Granada-Base Aérea: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30

Ejercicio 11:
Porcentaje: 2.44%


##Bloque 4: Operaciones avanzadas

In [142]:
#Ejercicio 12. Normalización de datos
#Normaliza cada variable al rango [0, 1] utilizando la fórmula:
#x_norm = (x - min) / (max - min). Aplica la normalización a cada variable
#de forma independiente, usando broadcasting.
min = np.min(datos, axis=(0, 1, 2))
max = np.max(datos, axis=(0, 1, 2))
datos_normalizados = (datos - min) / (max - min)

print(f"\nEjercicio 12:")
print("Rango original - min:", np.min(datos))
print("Rango original - max:", np.max(datos))
print("Rango normalizado - min:", np.min(datos_normalizados))
print("Rango normalizado - max:", np.max(datos_normalizados))

#Ejercicio 13. Anomalías térmicas
#Calcula la anomalía térmica de cada medición respecto a la media
#de su estación. ¿Qué medición presenta la mayor anomalía positiva?
print(f"\nEjercicio 13:")
media_por_estacion = np.mean(datos[:,:,:,0], axis=(1, 2))
anomalia = datos[:,:,:,0] - media_por_estacion.reshape(n_estaciones, 1, 1)
max_anomalia = np.max(anomalia)
print(f"Supera la temperatura media por {max_anomalia:.2f}°C")


#Ejercicio 14. Correlación temperatura-humedad
#Calcula el coeficiente de correlación entre temperatura y humedad
#para cada estación.
#¿Es la correlación positiva o negativa? ¿Tiene sentido físico?
print(f"\nEjercicio 14:")
for i in range(n_estaciones):
    temperatura = datos[i, :, :, 0].flatten()
    humedad = datos[i, :, :, 1].flatten()
    correlacion = np.corrcoef(temperatura, humedad)[0, 1] #[1, 0] vale
    print(f"{stations[i]}: Correlación = {correlacion:.4f}")


#Ejercicio 15. Potencial de enegía solar
#Calcula la energía solar diaria acumulada (suma de radiación por hora)
#para cada estación y día. ¿Qué estación tiene mayor potencial solar?
#Crea un ranking de estaciones por enegía solar media diaria.
energia_solar_diaria = np.sum(datos[:, :, :, 5], axis=2)
energia_media_estacion = np.mean(energia_solar_diaria, axis=1)
ranking = np.argsort(energia_media_estacion)[::-1]
print(f"\nEjercicio 15:")
for pos, est in enumerate(ranking):
    print(f"{pos + 1}º {stations[pos]}: {energia_media_estacion[est]:.2f} W/m²")

print(f"\nEstación con mayor potencial solar: {stations[ranking[0] + 1]}")



Ejercicio 12:
Rango original - min: -4.189024398926728
Rango original - max: 1029.8869145954131
Rango normalizado - min: 0.0
Rango normalizado - max: 1.0

Ejercicio 13:
Supera la temperatura media por 11.39°C

Ejercicio 14:
Madrid-Retiro: Correlación = -0.3732
Barcelona-El Prat: Correlación = -0.3602
Sevilla-Aeropuerto: Correlación = -0.3812
Bilbao-Sondica: Correlación = -0.3448
Granada-Base Aérea: Correlación = -0.4117

Ejercicio 15:
1º Madrid-Retiro: 3333.65 W/m²
2º Barcelona-El Prat: 3305.53 W/m²
3º Sevilla-Aeropuerto: 3305.02 W/m²
4º Bilbao-Sondica: 3291.87 W/m²
5º Granada-Base Aérea: 3279.58 W/m²

Estación con mayor potencial solar: Barcelona-El Prat


##Bloque 5: Ejercicio integrador

In [154]:
#Ejercicio 16. Informe meteorológico automatizado
#Crea una función que reciba el array de datos y genere un informe con las
#siguientes métricas para cada estación:
  #Temperatura: media, mínima, máxima, amplitud térmica media diaria
  #Precipitación: total acumulada, número de días con lluvia (>1 mm)
  #Viento: velocidad media, porcentaje de horas con viento fuerte (>40 km/h)
  #Radiación: energía total acumulada en el mes

#El informe debe devolver un array estructurado o un diccionario con
#los resultados. No uses bucles: todas las operaciones deben ser vectorizadas.
def informe_meteorologico(datos, stations):
    # Temperatura
    temp_media = np.mean(datos[:, :, :, 0], axis=(1, 2))
    temp_min = np.min(datos[:, :, :, 0], axis=(1, 2))
    temp_max = np.max(datos[:, :, :, 0], axis=(1, 2))

    # Amplitud térmica
    temp_max_diaria = np.max(datos[:, :, :, 0], axis=2)
    temp_min_diaria = np.min(datos[:, :, :, 0], axis=2)
    amplitud_termica = np.mean(temp_max_diaria - temp_min_diaria, axis=1)

    # Precipitación
    precip_total = np.sum(datos[:, :, :, 4], axis=(1, 2))
    precip_diaria = np.sum(datos[:, :, :, 4], axis=2)
    dias_lluvia = np.sum(precip_diaria > 0, axis=1)

    # Viento
    velocidad_media = np.mean(datos[:, :, :, 3], axis=(1, 2))
    horas_viento_fuerte = np.sum(datos[:, :, :, 3] > 40, axis=(1, 2))
    porcen_viento_fuerte = (horas_viento_fuerte / (n_dias * n_horas)) * 100

    # Radiación
    radiacion_total = np.sum(datos[:, :, :, 5], axis=(1, 2))

    return {
      'temp_media': temp_media,
      'temp_min': temp_min,
      'temp_max': temp_max,
      'amplitud_termica': amplitud_termica,
      'precip_total': precip_total,
      'dias_lluvia': dias_lluvia,
      'viento_medio': velocidad_media,
      'porc_viento_fuerte': porcen_viento_fuerte,
      'radiacion_total': radiacion_total
    }

informe = informe_meteorologico(datos, stations)
for i in range(n_estaciones):
    print(f"\n{stations[i]}:")
    print(f"  Temperatura media: {informe['temp_media'][i]:.2f}°C")
    print(f"  Temperatura mínima: {informe['temp_min'][i]:.2f}°C")
    print(f"  Temperatura máxima: {informe['temp_max'][i]:.2f}°C")
    print(f"  Amplitud térmica media diaria: {informe['amplitud_termica'][i]:.2f}°C")
    print(f"  Precipitación total: {informe['precip_total'][i]:.2f} mm")
    print(f"  Días con lluvia (>1mm): {informe['dias_lluvia'][i]}")
    print(f"  Viento medio: {informe['viento_medio'][i]:.2f} km/h")
    print(f"  Porcentaje viento fuerte (>40 km/h): {informe['porc_viento_fuerte'][i]:.2f}%")
    print(f"  Radiación total acumulada: {informe['radiacion_total'][i]:.2f}")



Madrid-Retiro:
  Temperatura media: 7.98°C
  Temperatura mínima: -2.81°C
  Temperatura máxima: 18.27°C
  Amplitud térmica media diaria: 14.81°C
  Precipitación total: 1536.97 mm
  Días con lluvia (>1mm): 30
  Viento medio: 15.22 km/h
  Porcentaje viento fuerte (>40 km/h): 0.42%
  Radiación total acumulada: 100009.54

Barcelona-El Prat:
  Temperatura media: 12.18°C
  Temperatura mínima: 2.56°C
  Temperatura máxima: 22.09°C
  Amplitud térmica media diaria: 14.29°C
  Precipitación total: 1531.75 mm
  Días con lluvia (>1mm): 30
  Viento medio: 15.88 km/h
  Porcentaje viento fuerte (>40 km/h): 0.83%
  Radiación total acumulada: 99150.70

Sevilla-Aeropuerto:
  Temperatura media: 14.05°C
  Temperatura mínima: 4.26°C
  Temperatura máxima: 25.11°C
  Amplitud térmica media diaria: 14.50°C
  Precipitación total: 1432.05 mm
  Días con lluvia (>1mm): 30
  Viento medio: 15.38 km/h
  Porcentaje viento fuerte (>40 km/h): 0.69%
  Radiación total acumulada: 99165.95

Bilbao-Sondica:
  Temperatura media